In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("heart.csv")
df = pd.get_dummies(df)
df.info()
X = df.drop("HeartDisease", axis=1).copy()
y = df["HeartDisease"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=369)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    int64  
 1   RestingBP          918 non-null    int64  
 2   Cholesterol        918 non-null    int64  
 3   FastingBS          918 non-null    int64  
 4   MaxHR              918 non-null    int64  
 5   Oldpeak            918 non-null    float64
 6   HeartDisease       918 non-null    int64  
 7   Sex_F              918 non-null    bool   
 8   Sex_M              918 non-null    bool   
 9   ChestPainType_ASY  918 non-null    bool   
 10  ChestPainType_ATA  918 non-null    bool   
 11  ChestPainType_NAP  918 non-null    bool   
 12  ChestPainType_TA   918 non-null    bool   
 13  RestingECG_LVH     918 non-null    bool   
 14  RestingECG_Normal  918 non-null    bool   
 15  RestingECG_ST      918 non-null    bool   
 16  ExerciseAngina_N   918 non

In [4]:
model = AdaBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8804347826086957

In [8]:
def objective(trial):
    hiper = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0, 1)
    }
    model = AdaBoostClassifier(**hiper)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_jobs=-1, n_trials=1000)

[I 2025-08-22 18:36:19,291] A new study created in memory with name: no-name-d996e5fd-22c5-4a02-96d0-af43df6f86df
[I 2025-08-22 18:36:20,721] Trial 6 finished with value: 0.8913043478260869 and parameters: {'n_estimators': 56, 'learning_rate': 0.8069421729524073}. Best is trial 6 with value: 0.8913043478260869.
[I 2025-08-22 18:36:21,439] Trial 15 finished with value: 0.8913043478260869 and parameters: {'n_estimators': 85, 'learning_rate': 0.3279350160828901}. Best is trial 6 with value: 0.8913043478260869.
[I 2025-08-22 18:36:21,606] Trial 10 finished with value: 0.8369565217391305 and parameters: {'n_estimators': 90, 'learning_rate': 0.01579594031003151}. Best is trial 6 with value: 0.8913043478260869.
[I 2025-08-22 18:36:22,285] Trial 16 finished with value: 0.8840579710144928 and parameters: {'n_estimators': 66, 'learning_rate': 0.908243024789989}. Best is trial 6 with value: 0.8913043478260869.
[I 2025-08-22 18:36:23,214] Trial 11 finished with value: 0.8913043478260869 and parame

In [15]:
best_hiper = study.best_params
print(best_hiper)
model = AdaBoostClassifier(**best_hiper)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)


{'n_estimators': 236, 'learning_rate': 0.37720320567001}
0.9021739130434783
